In [1]:
# @markdown  ### reload custom modules imported during runtime
# https://stackoverflow.com/questions/50339549/google-colab-reload-imported-modules

%load_ext autoreload
%autoreload 2

In [2]:
#@markdown ### Load custom environment as module into content folder.

import os
if not os.path.exists('NMA-group1'):
    !git clone -b impl_surya https://github.com/JuliaY123/NMA-group1.git
!cp '/content/NMA-group1/nback_env.py' '/content/nback_env.py'


from IPython.display import clear_output
clear_output()


Cloning into 'NMA-group1'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 134 (delta 32), reused 31 (delta 28), pack-reused 96
Receiving objects: 100% (134/134), 1.66 MiB | 3.70 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [5]:
#@markdown ### Supress warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
#@title Install dependencies
!pip install swig
!pip install gymnasium['all']

import gymnasium as gym
from gymnasium import spaces
import math
import numpy as np
from numpy.random import default_rng
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output

clear_output()


In [ ]:
# @title Define environment
# # N-back environment

# import numpy as np
# import gymnasium as gym
# from gymnasium import Env, spaces, utils

# class NBack(Env):

#     # Examples
#     # N = 2
#     # step_count =        [ 0  1   2  3  4  5  6 ]
#     # sequence =          [ a  b   c  d  a  d  a ] (except these are usually digits between 0-9)
#     # correct actions =   [ ~  ~   0  0  0  1  1 ]
#     # actions =           [ ~  ~   1  0  0  1  0 ]
#     # reward_class =      [ ~  ~  FP TN TN TP FN ]
#     # reward =            [ ~  ~  -1  0  0  1 -1 ]
#     # Rewards input is structured as (TP, TN, FP, FN) (positive being matches)

#     def __init__(self, N=2, num_trials=25, num_targets=None, rewards=(1, 1, -1, -1), num_obs=5, seed=2023):

#         self.N = N
#         self.num_trials = num_trials
#         self.episode_length = num_trials + self.N
#         self.num_targets = num_targets
#         self.rewards = rewards
#         self.num_obs = num_obs
#         self.num_actions = 2
#         # super().reset(seed=seed)

#         # Check that parameters are legal
#         assert(len(rewards) == 4)
#         assert(num_targets is None or num_targets <= num_trials)

#         # Define rewards, observation space and action space
#         self.reward_range = (min(rewards), max(rewards))    # Range of rewards based on inputs
#         # self.observation_space = spaces.Tuple([spaces.Discrete(10) for i in range(self.num_obs)])     # Tuple num_obs long with 10 possibilities
#         self.observation_space = spaces.Box(low=0, high=9, shape=(5, ))
#         self.action_space = spaces.Discrete(self.num_actions)                        # 0 (No match) or 1 (Match)

#     def reset(self, seed=None):

#         # Seed RNG
#         super().reset(seed=seed)

#         # Generate sequence and correct actions
#         self._generate_sequence()
#         self._get_correct_actions()

#         # Observation is first character
#         self.step_count = 0

#         # initialize
#         observation = self._get_obs()
#         info = self._get_info()

#         return observation, info

#     def step(self, action):

#         # Calculate reward
#         if self.step_count >= self.N:
#             if (self.correct_actions[self.step_count - self.N]): # Match
#                 reward = self.rewards[0] if action else self.rewards[3] # TP if matched else FN
#             else: # No match
#                 reward = self.rewards[2] if action else self.rewards[1] # FP if matches else TN
#         else:
#             reward = 0

#         # Return next character or None

#         self.step_count += 1
#         observation = self._get_obs()
#         info = self._get_info()

#         if self.step_count < self.episode_length:
#             return observation, reward, False, info
#         else:
#             return observation, reward, True, info

#     def _generate_sequence(self):

#         # Generate sequence of length self.episode_length (with correct number of targets)
#         while True:
#             self.sequence = np.random.randint(0, 9, size=(self.episode_length))
#             if not self.num_targets or sum(self._get_correct_actions()) == self.num_targets:
#                 break


#     def _get_obs(self):

#         if self.step_count < self.num_obs:
#             window = self.sequence[:self.step_count + 1]
#             observation = np.pad(window, (self.num_obs - self.step_count -1, 0), mode='constant', constant_values=(0))
#         elif self.step_count == self.episode_length:
#             window = self.sequence[self.step_count + 1 - self.num_obs : self.step_count + 1]
#             observation = np.pad(window, (0,1), mode='constant', constant_values=(0))
#         else:
#             window = self.sequence[self.step_count + 1 - self.num_obs : self.step_count + 1]
#             observation = window

#         return observation

#     def _get_correct_actions(self):
#         self.correct_actions = np.array([int(self.sequence[i] == self.sequence[i + self.N]) for i in range(self.num_trials)])
#         return self.correct_actions

#     def _get_info(self):
#         info = {
#             'step_count': self.step_count,
#             }
#         return info



In [ ]:
#@title Random Agent

import gymnasium as gym
from gymnasium import register

register(
    id='NBack-v0',
    entry_point='nback_env:NBack',
)

env = gym.make(
    'NBack-v0',
    N = 3,
    num_trials=100,
    num_targets=10,
    rewards=(1, 0, 0, 0),
    num_obs=5,
    seed=2023
    )

observation, info = env.reset()
print(f"reset observation:\t{observation}\n")

done = False

while not done:
    action = env.action_space.sample()
    next_observation, reward, done, info = env.step(action)
    print(f"observation:\t{observation}")
    print(f"action:\t{action}")
    print(f"reward:\t{reward}")
    print(f"next_observation:\t{next_observation}")
    print(f"done:\t{done}")
    print(f"info:\t{info['step_count']}")
    print(f"\n")
    observation = next_observation

In [48]:
#@title DQN Agent with Experience Replay

import random

class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, next_state, reward, done):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, next_state, reward, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        states, actions, next_states, rewards, dones = zip(*batch)
        return np.stack(states), actions, np.stack(next_states), rewards, dones

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [ ]:
#@title DQN with Recurrent Networks and Experience Replay

import random

class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, next_state, reward, done):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, next_state, reward, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        states, actions, next_states, rewards, dones = zip(*batch)
        return np.stack(states), actions, np.stack(next_states), rewards, dones

    def __len__(self):
        return len(self.memory)

class DRQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DRQN, self).__init__()
        self.rnn1 = nn.RNN(input_size, 64, 2, batch_first=True)
        self.fc1 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        r_out, hidden = self.rnn1(x)
        r_out = r_out.view(-1, 64)
        x = self.relu(self.rnn1(x))
        x = self.softmax(self.fc1(x))
        return x


In [49]:
def train(model, memory, optimizer, criterion, batch_size, gamma):
    if len(memory) < batch_size:
        return
    states, actions, next_states, rewards, dones = memory.sample(batch_size)
    states = Variable(torch.FloatTensor(states))
    actions = Variable(torch.LongTensor(actions))
    next_states = Variable(torch.FloatTensor(next_states))
    rewards = Variable(torch.FloatTensor(rewards))
    dones = Variable(torch.FloatTensor(dones))

    q_values = model(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    next_q_values = model(next_states).max(1)[0]

    target_q_values = rewards + gamma * next_q_values * (1 - dones)

    loss = criterion(q_values, target_q_values.detach())
    loss_value = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss_value


def exec_training(
    env,
    gamma=0.99,
    num_episodes = 100,
    capacity = 1000,
    batch_size = 64,
    lr = 0.001
    ):

    # Create an instance of the DQN model
    input_size = env.observation_space.shape[0]
    output_size = env.action_space.n
    model = DQN(input_size, output_size)

    # Create an instance of the replay memory
    memory = ReplayMemory(capacity)

    # Set up the optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    rewards_list = []
    loss_list = []

    # Training loop
    for episode in range(num_episodes):
        state = env.reset()
        state = state[0]
        done = False

        total_reward = 0
        episode_loss = []
        step_count = 1

        while not done:
            # Select an action using epsilon-greedy policy
            epsilon = max(0.01, 0.08 - 0.01 * episode)
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                with torch.no_grad():
                    q_values = model(torch.FloatTensor(state))
                    action = q_values.argmax().item()

            # Take the selected action and observe the next state and reward
            # next_state, reward, done, terminated, truncated = env.step(action)
            next_state, reward, done, info = env.step(action)

            # Store the transition in the replay memory
            memory.push(state, action, next_state, reward, done)

            # Move to the next state
            state = next_state
            total_reward += reward

            # Train the model
            loss = train(model, memory, optimizer, criterion, batch_size, gamma)

            if loss is None:
                loss = 0
            episode_loss.append(loss)

            step_count += 1

        rewards_list.append(total_reward)
        loss_list.append(np.mean(episode_loss))

        # Print the total reward for the episode
        # print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    return rewards_list, loss_list




In [52]:
# Create environment, random agent and test

import gymnasium as gym
from gymnasium import register

SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

register(
    id='NBack-v0',
    entry_point='nback_env:NBack',
)

env = gym.make(
    'NBack-v0',
    N = 2,
    num_trials=100,
    num_targets=20,
    rewards=(1, 0, -1, -1),
    num_obs=5,
    seed=SEED
    )

observation, info = env.reset()
rewards, losses = exec_training(env=env, gamma=0.99, num_episodes = 1)

0 0
0 0
1 0
2 0
3 1
4 1
5 0
6 0
7 0
8 0
9 1
10 0
11 1
12 0
13 1
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 1
32 0
33 1
34 0
35 1
36 0
37 0
38 0
39 0
40 1
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 1
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 1
61 0
62 0
63 0
64 0
65 1
66 0
67 1
68 0
69 0
70 0
71 1
72 0
73 1
74 0
75 1
76 1
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 1
87 1
88 0
89 0
90 0
91 0
92 0
93 1
94 0
95 0
96 0
97 0
98 0
99 0
100 0


IndexError: ignored

In [46]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

fig = make_subplots(rows=2, cols=1)
# fig = go.Figure()

def generate_figure_traces(data_dict):

    y = data_dict['data']
    label = data_dict['label']
    # Create the data sequence
    y = np.array(y)  # Example y values for line 1
    x = np.arange(y.shape[0])  # Example x values

    # Create the line trace
    trace = go.Scatter(x=x, y=y, mode='lines', name=label)

    return trace

data = [
    {'data': rewards, 'label': 'reward'},
    {'data': losses, 'label': 'loss'},
    ]

traces = list(map(lambda x: generate_figure_traces(x), data))

for index, trace in enumerate(traces):
    # add the first plot
    fig.add_trace(
        trace,
        row=index+1, col=1
    )

    # Create the layout
    fig.update_layout(
        title='Metrics Plot',
    )

# Show the plot
fig.show()
